In [1]:
import pandas as pd
import re
import openpyxl 
import numpy as np
from timeit import default_timer as timer
from datetime import datetime
import os

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
#predefine parameters
filterTransDesc = "HANDLING CHRG"

#name col to remove rows with missing na or to match bs
payoutNameCol = 'Member Name'
merchantNameCol = 'Merchant'
merchantBankNameCol = 'Bank Acc Name'
billPaymentNameCol = "Name"

#col of payout to match amount (payout, merchant and bill payment must have)
poAmountCol = "Amount"
#col of final output as name
poNameCol = "Member Name"
bsName = "BS Name"

#cols to match in bank statement
# bsColMatchList = ['Reference 1', 'Reference 2', 'Reference 3', 'Reference 4']
bsMatchCol = "Reference 2"
bsAmountCol = "Debit Amount"

#index col to track rows
euIndex = "EU_Index"
copIndex = "COP_Index"
manageCOPIndex = "ManageCOP_Index"
merchantIndex = "Merchant_Index"
billPaymentIndex = "BP_Index"
bsIndex = "BS_Index"

#ruleDict, which file type to use which set of rules
ruleDict = {"General":[1, 2, 3, 4, 6],
           "Merchant":[1, 2, 3, 4],
           "BillPayment":[2, 5]}

payoutExcelFile = "Quinton_Payout_202011/PAYOUT - 03.11.2020 .xlsx"
# payoutExcelFile = "Quinton_Payout_202012/PAYOUT - 04.12.2020.xlsx"

#get date
date = re.search('^.*?(\d+\.\d+)',payoutExcelFile.split('-')[1]).group(1)
date = datetime.strptime(date, '%d.%m')
date

datetime.datetime(1900, 11, 3, 0, 0)

In [3]:
#get worksheet names
wb = openpyxl.load_workbook(payoutExcelFile)
wsList = wb.sheetnames

EU_ws = [i for i in wsList if re.search('EU\s*$', i)][0]
#preceding cannot be manage
COP_ws = [i for i in wsList if re.search('(?<!MANAGE[\s\-])COP\s*$', i)][0]
ManageCOP_ws = [i for i in wsList if re.search('MANAGE COP\s*$', i)][0]
Merchant_ws = [i for i in wsList if re.search('MERCHANT[^-]', i)][0]
BillPayment_ws = [i for i in wsList if re.search('BILL PAYMENT[^-]*\s*$', i)][0]
print("EU WS - %s"%EU_ws)
print("COP WS - %s"%COP_ws)
print("ManageCOP WS - %s"%ManageCOP_ws)
print("Merchant WS - %s"%Merchant_ws)
print("BillPayment WS - %s"%BillPayment_ws)

#read file

#read payout files - EU, COP and ManageCOP
dfEU = pd.read_excel(payoutExcelFile, sheet_name = EU_ws)
#remove empty member name
dfEU = dfEU[pd.notnull(dfEU[payoutNameCol])]
dfEU['DataType'] = "EU" 

dfCOP = pd.read_excel(payoutExcelFile, sheet_name = COP_ws)
#remove empty member name
dfCOP = dfCOP[pd.notnull(dfCOP[payoutNameCol])]
dfCOP['DataType'] = "COP" 

dfManageCOP = pd.read_excel(payoutExcelFile, sheet_name = ManageCOP_ws)
#remove empty member name
dfManageCOP = dfManageCOP[pd.notnull(dfManageCOP[payoutNameCol])]
dfManageCOP['DataType'] = "ManageCOP" 

#merchant file
dfMerchant = pd.read_excel(payoutExcelFile, sheet_name = Merchant_ws)
#remove empty merchant col
dfMerchant = dfMerchant[pd.notnull(dfMerchant[merchantNameCol])]
dfMerchant['DataType'] = "Merchant" 

dfBillPayment = pd.read_excel(payoutExcelFile, sheet_name = BillPayment_ws)
#remove empty name
dfBillPayment = dfBillPayment[pd.notnull(dfBillPayment[billPaymentNameCol])]
dfBillPayment['DataType'] = "BillPayment" 

#create index
dfEU[euIndex] = np.arange(dfEU.shape[0])
dfCOP[copIndex] = np.arange(dfCOP.shape[0])
dfManageCOP[manageCOPIndex] = np.arange(dfManageCOP.shape[0])
dfMerchant[merchantIndex] = np.arange(dfMerchant.shape[0])
dfBillPayment[billPaymentIndex] = np.arange(dfBillPayment.shape[0])

EU WS - 26.10- EU
COP WS - 28-29.10-COP
ManageCOP WS - 28-19.10-MANAGE COP
Merchant WS - 28-29.10-MERCHANT 
BillPayment WS - 30.10-01.11-BILL PAYMENT 


In [4]:
if date.month == 11:
    bsFolder = 'Nov2020_Result'
    outputFolder = "Nov2020_MatchingResult"
elif date.month == 12:
    bsFolder = "Dec2020_Result"
    outputFolder = "Dec2020_MatchingResult"
bsDatePattern = date.strftime('%d_%m')
bsExcelFile = [i for i in os.listdir(bsFolder) if re.search(bsDatePattern, i)][0]
print("Output filename: %s"%bsExcelFile)

# dfBankStatement = pd.read_excel(bsFolder + '/' + bsExcelFile, skiprows = 9)
dfBankStatement = pd.read_excel("03_11 3207971832_2020_addRef.xlsx", skiprows = 9)

#filter out ref1 gas 
filterTransDesc = "HANDLING CHRG"
dfBankStatement2 = dfBankStatement[dfBankStatement[bsMatchCol].map(lambda x:
                                                    False if re.search(filterTransDesc, str(x))
                                                           else True)].copy()

#create bsIndex
dfBankStatement2[bsIndex] = np.arange(dfBankStatement2.shape[0])
#keep track bsIndex list
bsIndexList = dfBankStatement2[bsIndex].unique()

Output filename: 03_11 3207971832_2020.xlsx


In [62]:
bsIndexList = dfBankStatement2[bsIndex].unique()

In [42]:
def ExactMatch(df, indexCol, nameCol, dfBS, dtype):
    startTime = timer()
    nameList = [str(i).lower() for i in df[nameCol].values]
    amountList = df[poAmountCol].values
    indexList = df[indexCol].values
    matchResult = []
    for no, name in enumerate(nameList):
        POindex = indexList[no]
        POname = nameList[no]
        POamount = amountList[no]
        dfTemp = dfBS[(dfBS[bsMatchCol].map(lambda x:name in x.lower() and x.lower()!='nan'))
                        & (dfBS[bsAmountCol] == amountList[no])].copy()
        if dfTemp.shape[0] >= 1:
            bsAmount = dfTemp[bsAmountCol].values[0]
            bsMatchIndex = dfTemp[bsIndex].values[0]
            bsName = dfTemp[bsMatchCol].values[0]
            #[POindex, POname, POamount, BSindex, BSname, BSamount]
            matchResult.append([POindex, POname, POamount,
                               bsMatchIndex, bsName, bsAmount])
            
#         elif dfTemp.shape[0] > 1:
#             for no2, name in enumerate(dfTemp[bsMatchCol].values):
#                 BSname = name
#                 BSamount = dfTemp[bsAmountCol].values[no2]
#                 BSindex = dfTemp[bsIndex].values[no2]
#                 NoRow = list(dfBS[bsIndex]).index(BSindex)
#                 #nid to remove duplicate after consolidation
#                 matchResult.append([POindex, POname, POamount, 
#                                     BSindex, BSname, BSamount])
    #form table
    colList = [indexCol, nameCol, poAmountCol, 
               bsIndex, bsMatchCol, bsAmountCol]
    dfResult = pd.DataFrame(matchResult, columns = colList).copy()
    dfResult['MatchCategory'] = "ExactMatch"
    dfResult['DataType'] = dtype
    dfResult.rename(columns = {nameCol: payoutNameCol}, inplace = True)
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return dfResult

In [121]:
dfEU.query("EU_Index == 1275")

,Country,Withdraw Date,Username,Member Name,Withdraw,Package,Bank,Bank Acc,Bank-in Slip,Bank-in By,Bank-in Date,Remarks,Status,Amount,Unnamed: 14,Unnamed: 15,DataType,EU_Index
1275,MALAYSIA,2020-10-26,nozie1430,NOOR ZILFADZLINA BINTI MAMAT,EU,EU,CIMB-MALAYSIA,7048697132,NaN,NaN,NaN,NaN,Pending,499.0,Edit,Reject,EU,1275


In [119]:
dfEU.query("EU_Index == 1275")['Member Name'].values[0]

'NOOR ZILFADZLINA BINTI MAMAT'

In [122]:
dfBankStatement2.query("BS_Index == 4373")

,Trn. Date,Reference 1,Reference 2,Debit Amount,Credit Amount,Balance,BS_Index
6350,03/11,MISC DR 0028193207971832PM03112099 EU REFUNDEU...,MISC DR 0028193207971832PM03112099 EU REFUNDEU...,499.0,NaN,3.25766e+07,4373


In [107]:
def matchFirst2Terms(name, series):
    ruleName = "matchFirst2Terms"
    
    #match series with first 1 or 2 terms of names
    if len(name.split(' ')) >= 3:
        match = ' '.join(name.split(' ')[:2])
        resultArray = series.map(lambda x:ruleName if re.search('[\s\d]%s'%re.escape(match), 
                                                                re.sub("\xa0", "", str(x)), flags = re.IGNORECASE)
                                else "False").values
    elif len(name.split(' ')) == 2:
        match = name.split(' ')[0]
        resultArray = series.map(lambda x:ruleName if re.search('[\s\d]%s'%re.escape(match), 
                                                                re.sub(" |\xa0","",str(x)), flags = re.IGNORECASE)
                                else "False").values
    else:
        resultArray = np.full(len(series), "False")

    return resultArray

# def matchTruncated(name, series):
#     ruleName = "matchTruncated"
#     #match series with truncated name if there is 
#     cleanName = re.sub(" binti ", " bt ", str(name), flags = re.IGNORECASE)
#     rule1 =  series\
#             .map(lambda x: True if re.search('%s'%re.escape(str(name)), str(x), flags = re.IGNORECASE) 
#                  and str(x) not in ['', 'nan'] else False).values
#     #match name with truncated name in the series if there is
#     rule2 = series\
#                 .map(lambda x: True if re.search('%s'%re.escape(re.sub("ENCIK |COP REFUND", '', 
#                                                                 re.sub(" binti ", " bt ", str(x), flags = re.I), 
#                                                                        flags = re.IGNORECASE).strip()), 
#                                                  str(name), flags = re.IGNORECASE) 
#                      and re.sub("ENCIK |COP REFUND", '', 
#                                 re.sub(" binti ", " bt ", str(x), flags = re.I), 
#                                 flags = re.IGNORECASE).strip() not in ['', 'nan'] else False).values
#     resultArray = (rule1|rule2)
#     resultArray = np.where(resultArray, ruleName, "False")
    
#     return resultArray

#for merchant only
def matchTruncated(name, series):
    ruleName = "matchTruncated"
    #match truncated name series with name if there is 
    rule1 = []
    for value in series:
        if re.search('.*\d\s*(.+)$', str(value)):
            truncatedName = re.escape(re.search('.*\d\s*(.+)$', str(value)).group(1))
            if truncatedName not in ['\ ', 'nan'] and re.search(truncatedName, name, flags = re.IGNORECASE):
                rule1.append(True)
            else:
                rule1.append(False)
        else:
            rule1.append(False)
    resultArray = np.where(rule1, ruleName, "False")
    return resultArray

def noSpaceMatch(name, series):
    ruleName = "noSpaceMatch"
    cleanName = re.sub(" ", "", str(name))
    #remove space for BS reference to match wth name
    rule1 = series.map(lambda x:True if re.search('%s'%re.escape(cleanName), 
                                                  re.sub(" ", "", x), flags = re.IGNORECASE)
                                    and cleanName not in ['', 'nan']
                                else False).values
#     rule2 = series.map(lambda x:True if re.search('%s'%re.escape(re.sub(' ','',re.sub("COP REFUND", '',str(x)))), 
#                         str(name), flags = re.IGNORECASE) and re.sub(' ','',re.sub("COP REFUND", '',str(x)))!=''
#                             else False).values
#     resultArray = (rule1|rule2)
    resultArray = rule1
    resultArray = np.where(resultArray, ruleName, "False")
    return resultArray

def matchAccountLast4Digit(accountNo, series):
    ruleName = "matchAccountLast4Digit"
    resultArray = series.map(lambda x:ruleName if re.search('%s'%accountNo, str(x), flags = re.IGNORECASE)
                                        and len(accountNo) == 4 else "False").values
    return resultArray

def matchBillTypeRef(billType, series):
    ruleName = "matchBillTypeRef"
    if billType == '':
        resultArray = np.full(len(series), "False")
        reason = ""
    else:
        #if can find billtype - Indah water in reference 
        rule1 =  series\
                .map(lambda x: True if re.search('%s'%billType, str(x), flags = re.IGNORECASE) 
                     and str(x) not in  ['', 'nan'] else False).values
        #if billtype is PAIP and Pengurusan air pahan in reference
        rule2 = series\
                .map(lambda x: True if re.search('%s'%"PENGURUSAN AIR PAHAN", str(x), flags = re.IGNORECASE) 
                     and billType == 'PAIP' else False).values
        #if billtype is PBA and perbadanan bekalan air in reference
        rule3 = series\
                .map(lambda x: True if re.search('%s'%"perbadanan bekalan air|perbadanan bekalan", str(x), 
                                                 flags = re.IGNORECASE) 
                     and billType == 'PBA' else False).values
        resultArray = (rule1|rule2|rule3)

        resultArray = np.where(resultArray, ruleName, "False")
    return resultArray

def matchUniqueAmount(amount, series):
    ruleName = "matchUniqueAmount"
    #if only unique match in bs for amount
    resultArray = series.map(lambda x:True if x == amount else False).values
    if sum(resultArray) == 1:
        return np.full(len(series), ruleName)
    else:
        return np.full(len(series), "False")

def PartialMatch(df, indexCol, notMatchIndex, nameCol, dfBS, ruleType):
    startTime = timer()
    matchResult = []
    bsIndexCheckList = []
    df = df[df[indexCol].isin(notMatchIndex)].copy()
    for no, row in enumerate(list(df.iterrows())):
        #assign values to variables: POname, POamount, AccountNo, BillType
        index = row[1][indexCol]
        POname = row[1][nameCol]
        POname = re.sub("\u200b", "", str(POname))
        POname = re.sub("\xa0", "", str(POname))
        POamount = row[1][poAmountCol]
        print(index)
        #merchant, other payout
        if "Bank Acc" in df.columns:
            AccountNo = re.sub(' |-|\.0|\(|\)','', str(row[1]['Bank Acc']))[-4:]
            match4 = matchAccountLast4Digit(AccountNo, dfBS[bsMatchCol])
        #bill payment
        elif "Acc Number" in df.columns:
            AccountNo = re.sub(' |-|\.0|\(|\)','', str(row[1]['Acc Number']))[-4:]
            match4 = matchAccountLast4Digit(AccountNo, dfBS[bsMatchCol])
        else:
            AccountNo = ''
            
        if 'Bill Type' in df.columns:
            BillType = row[1]['Bill Type']
            match5 = matchBillTypeRef(BillType, dfBS[bsMatchCol])
        else:
            BillType = ''
        
        #matching rules
        if ruleType in ["General", "Merchant"]:
            match1 = matchTruncated(POname, dfBS[bsMatchCol])
        match2 = matchFirst2Terms(POname, dfBS[bsMatchCol])
#         match3 = noSpacenoCopRefund(POname, dfBankStatement2[bsMatchCol])
        match3 = noSpaceMatch(POname, dfBS[bsMatchCol])
        match6 = matchUniqueAmount(POamount, dfBS[bsAmountCol])
        ruleStr = "|".join(['(match%s!="False")'%i for i in ruleDict[ruleType]])
        checkRule = eval(ruleStr)
        #assign bool statement to checkRule
        #use result of matching rules to subset dfBankStatement2
        dfMatch = dfBS[(dfBS[bsAmountCol] == POamount) & (checkRule)].copy()
        
        #to copy rulename if matches 
        ruleStr = '[' + ','.join(['match%s'%i for i in ruleDict[ruleType]]) + ']'
        #assign list to ruleList
        ruleList = eval(ruleStr)

        if dfMatch.shape[0] == 1:
            BSname = dfMatch[bsMatchCol].values[0]
            BSamount = dfMatch[bsAmountCol].values[0]   
            BSindex = dfMatch[bsIndex].values[0]
            NoRow = list(dfBS[bsIndex]).index(BSindex)
            print(BSname)
            print(BSindex)
            rule = ','.join([i[NoRow] for i in ruleList if i[NoRow]!="False"])
            matchResult.append([index, POname, POamount, BSindex, BSname, BSamount, rule])
            if BSindex not in bsIndexCheckList:
                bsIndexCheckList.append(BSindex)
                
        elif dfMatch.shape[0] > 1:
            for no, name in enumerate(dfMatch[bsMatchCol].values):
                BSindex = dfMatch[bsIndex].values[no]
                if BSindex not in bsIndexCheckList:
                    BSname = dfMatch[bsMatchCol].values[no]
                    print(BSname)
                    print(BSindex)
                    BSamount = dfMatch[bsAmountCol].values[no]
                    NoRow = list(dfBS[bsIndex]).index(BSindex)
                    rule = ','.join([i[NoRow] for i in ruleList if i[NoRow]!="False"])
                    #nid to remove duplicate after consolidation
                    matchResult.append([index, POname, POamount, BSindex, BSname, BSamount, rule])
                    bsIndexCheckList.append(BSindex)
                    break
                    
                
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return matchResult

def constructTable(matchResult, df, indexName, nameCol, dType):
    pandaList = []
    for i in matchResult:
        pandaList.append(i)
    
    dfMatchFinal = pd.DataFrame(pandaList)
    if dfMatchFinal.shape[0]!=0:
        #[index, POname, POamount, BSindex, BSname, BSamount, rule]
        dfMatchFinal.columns = [indexName, poNameCol, poAmountCol, bsIndex, bsMatchCol, bsAmountCol, 'Reason']
        dfMatchFinal['MatchCategory'] = "PartialMatch"
        dfMatchFinal['DataType'] = dType
        return dfMatchFinal
    else:
        return None

In [133]:
dfEU.query("EU_Index == 215")

,Country,Withdraw Date,Username,Member Name,Withdraw,Package,Bank,Bank Acc,Bank-in Slip,Bank-in By,Bank-in Date,Remarks,Status,Amount,Unnamed: 14,Unnamed: 15,DataType,EU_Index
215,MALAYSIA,2020-10-26,siochin,CH’NG SIO CHIN,EU,EU,MAYBANK BERHAD,159012827265,NaN,NaN,NaN,NaN,Pending,1200.0,Edit,Reject,EU,215


In [134]:
# dfEU.query("EU_Index == 1275")['Member Name'].values[0]

dfBankStatement2.query("BS_Index == 8381")

,Trn. Date,Reference 1,Reference 2,Debit Amount,Credit Amount,Balance,BS_Index
14475,03/11,INSTANT TRSF DR 701077 CH'NG SIO CHINEU REFUND,INSTANT TRSF DR 701077 CH'NG SIO CHINEU REFUND,1200.0,NaN,2.86299e+07,8381


In [132]:
"CH'NG SIO CHIN" in "INSTANT TRSF DR 701077 CH'NG SIO CHINEU REFUND "

True

In [44]:
#COP
#287
dfBankStatementTemp = dfBankStatement2[dfBankStatement2[bsIndex].isin(bsIndexList)].copy()
dfExactMatchCOP = ExactMatch(dfCOP, copIndex, payoutNameCol,
                             dfBankStatementTemp, "COP")
#find no match list
noMatchCOPList = set(dfCOP[copIndex].values)\
                    .difference(set(dfExactMatchCOP[copIndex].values))

#remove bs duplicates
# dfExactMatchCOP.drop_duplicates(copIndex, inplace = True)
dfExactMatchCOP.drop_duplicates(bsIndex, inplace = True)

print("Total row - BS before match: %s"%len(bsIndexList))
#update bsIndexList
bsIndexList = [i for i in bsIndexList if i not in dfExactMatchCOP['BS_Index'].unique()]
print("Total row - BS after match: %s"%len(bsIndexList))
print("Total row: %s"%dfCOP.shape[0])
print("Exact Matches: %s"%dfExactMatchCOP.shape[0])
print("No Matches: %s"%len(noMatchCOPList))

Total time: 3.5051s
Total row - BS before match: 8487
Total row - BS after match: 8200
Total row: 380
Exact Matches: 287
No Matches: 86


In [108]:
#84
bsIndexList = dfBankStatement2[bsIndex].unique()
# bsIndexList = [2137]
copList = [105, 301]
dfCOP2 = dfCOP.query("COP_Index in @copList").copy()
dfBankStatementTemp = dfBankStatement2[dfBankStatement2[bsIndex].isin(bsIndexList)].copy()
matchResultCOP = PartialMatch(dfCOP2, copIndex, noMatchCOPList, 
                              payoutNameCol, dfBankStatementTemp, ruleType = "General")

dfPartialMatchCOP = constructTable(matchResultCOP, dfCOP, copIndex, poNameCol, 'COP')

dfPartialMatchCOP.drop_duplicates(bsIndex, inplace = True)

print("Total row - BS before match: %s"%len(bsIndexList))
#update bsIndexList
bsIndexList = [i for i in bsIndexList if i not in dfPartialMatchCOP['BS_Index'].unique()]
print("Total row - BS after match: %s"%len(bsIndexList))
print("No Matches Previously: %s"%len(noMatchCOPList))
print("Partial Matches: %s"%dfPartialMatchCOP.shape[0])

105
DR-ECP 0006673207971832PM03112098 COP REFUNDCOP REFUND 29102020 CHEW ZHIANG CHOO
1980
301
DR-ECP 0007213207971832PM03112098 COP REFUNDCOP REFUND 28102020 CHEW ZHIANG CHOO
2137
Total time: 0.8174s
Total row - BS before match: 8487
Total row - BS after match: 8485
No Matches Previously: 86
Partial Matches: 2


In [109]:
dfPartialMatchCOP

,COP_Index,Member Name,Amount,BS_Index,Reference 2,Debit Amount,Reason,MatchCategory,DataType
0,105,CHEW ZHIANG CHOON,10000.0,1980,DR-ECP 0006673207971832PM03112098 COP REFUNDCO...,10000.0,"matchTruncated,matchFirst2Terms",PartialMatch,COP
1,301,CHEW ZHIANG CHOON,10000.0,2137,DR-ECP 0007213207971832PM03112098 COP REFUNDCO...,10000.0,"matchTruncated,matchFirst2Terms",PartialMatch,COP


In [81]:
301 in dfPartialMatchCOP['COP_Index'].values

False

In [82]:
dfCOP.query("COP_Index == 301")['Member Name'].values[0]

'CHEW ZHIANG CHOON'

In [92]:
dfBankStatement2.query("BS_Index == 2137")['Reference 2'].values[0]

'DR-ECP 0007213207971832PM03112098 COP REFUNDCOP REFUND 28102020 CHEW ZHIANG CHOO'

In [71]:
105 in dfPartialMatchCOP['COP_Index'].values

False

In [73]:
2137 in bsIndexList

True

In [72]:
1980 in bsIndexList

False

### Validation

In [ ]:
excelFile = "Nov2020_MatchingResult/03_11 3207971832_2020.xlsx"
dfValidation = pd.read_excel(excelFile, sheet_name = )